In [1]:
import collections
Individual = collections.namedtuple('Individual', ['Name', 'Marks', 'WebAssign', 'Exam', 'RawData'])
Marks = collections.namedtuple('Marks', ['Score', 'NumberOfNs'])

## Key      |   Data
## ---------+---------
## Name     |  full name:  last, first
## Marks    |  a namedtuple of type Marks
## WebAssign|  WebAssign score (whole number, max 100)
## Raw Data |  full pandas Series whose index is the Name above

###########################################

midtermOrFinal = "final"
#extra_text = "This is a check-in for your progress as of the middle of the semester. For the midterm grade, I'm reporting only the plain letter grade, no +/-.  Remember this is only to <i>help you</i> see where you stand at this point so you can know where to focus going forward."
extra_text = "<h2>Almost the end of classes</h2><p> <b>Note:</b> This is being prepared for the last week of classes.  Note also the grade thresholds that must be met on the next page (including WebAssign grades).</p> <p> Unhappy with your grade? There's still opportunities on Checkpoint 8 this week and 9 during the finals week. You also can work on every WebAssign assignment until May 5<p>Have you taken the final and are happy with your grade? High five 🙌. You're done 🎉. Your grade will not decrease. If you do nothing else, this is your final grade. (unless you don't take the final exam...)</p>"
# extra_text = "This is the final grade report. Letter grade with any plus/minus will be reported to MyWSU. Please don't email me about grades."

    

course = "math242"  #math242 or math243 as a string

The threshold variables are html tables which specify the required thresholds for getting a particular letter grade.

If the grading thresholds change (in a future semester, perhaps), adjust these tables in addition to the code.

In [2]:
midterm_144 = ("Through 3.5", # WebAssign sections
               21, 80,  #A 
               19, 2, 70, #B
               16, 4, 60, #C
               14, 50,  #D
               14, 50) #F

final_144 = ("(all)", # WebAssign sections
               48, 80,  #A 
               40, 2, 70, #B
               34, 6, 60, #C
               30, 50,  #D
               30, 50) #F

midterm_243 = ("(12.1-12.4 Chapter 6, Chapter 7, 8.1, 8.2)", # WebAssign sections
               20, 80,  #A 
               18, 2, 70, #B
               14, 4, 60, #C
               12, 50,  #D
               12, 50) #F

final_243 = ("(all)", # WebAssign sections
               44, 80,  #A 
               37, 2, 70, #B
               33, 6, 60, #C
               29, 50,  #D
               29, 50) #F

midterm_242 = ("(through section 3.5)", midtermOrFinal, # WebAssign sections
               21, 80, 9, #A    ## points; webassign; midterm marks
               19, 2, 70, 7, #B   ## points; below P; webassign; midterm marks
               16, 4, 60, 6, #C   ## points; below P; webassign; midterm marks
               14, 50, 5, #D   ## points;  below P;webassign; midterm marks
               14, 50) #F   ## points; webassign; midterm marks

final_242 = ("(all)", "Final", # WebAssign sections
               43, 80, 9, #A   ## points; webassign; midterm marks
               36, 2, 70, 7, #B  ## points; below P; webassign; midterm marks
               32, 6, 60, 6, #C  ## points; below P; webassign; midterm marks
               28, 50, 5, #D  ## points; below P; webassign; midterm marks
               28, 50) #F  ## points; webassign; 

midterm_123 = ("(through section 2.5)", # WebAssign sections
               24, 80,  #A 
               21, 2, 70, #B
               18, 4, 60, #C
               17, 50,  #D
               17, 50) #F

final_123 = ("(all)", # WebAssign sections
               44, 80,   #A 
               38, 2, 70, #B
               35, 6, 60, #C
               28, 50,  #D
               28, 50) #F

html_threshold_template = """<table class='paleGrayRows'> <thead> <tr> 
<th>Letter Grade</th> 
<th>Course Learning Targets</th> 
<th>WebAssign Homework %s</th> 
<th>%s Exam marks above N</th></tr> 
</thead> 
<tbody> 
<tr> <td>A</td> <td>≥ %d points and no scores below proficient</td> <td>≥ %d%%</td> <td>%d</td></tr> 
<tr> <td>B</td> <td>≥ %d points and at most %d scores below proficient</td> <td>≥ %d%%</td><td>%d</td> </tr> 
<tr> <td>C</td> <td>≥ %d points and at most %d scores below proficient</td> <td>≥ %d%%</td><td>%d</td> </tr> 
<tr> <td>D</td> <td>≥ %d points</td> <td>≥ %d%%</td> <td>%d</td> </tr> 
<tr> <td>F</td> <td>&lt; %d points</td> <td>&lt; %d%%</td> <td></td> </tr> 
</tbody> </table>"""



# I'm an idiot an still have this midterm vs final; oh well
threshold_table = final_242 # final_242 if course=="math242" else final_243


threshold_html_table = html_threshold_template % threshold_table


def decide_letter_grade(individual):

    ## If they haven't taken the final, replace the letter grade information...
    if individual.Exam >= 1000:
        return "Take the Final"

    threshold = { "A": {"Points": threshold_table[2], "Below P": 0, "WebAssign": threshold_table[3], "Exam marks": threshold_table[4]},
           "B": {"Points": threshold_table[5], "Below P": threshold_table[6], "WebAssign": threshold_table[7], "Exam marks": threshold_table[8]},
           "C": {"Points": threshold_table[9], "Below P": threshold_table[10], "WebAssign": threshold_table[11], "Exam marks": threshold_table[12]},
           "D": {"Points": threshold_table[13], "Below P": 1000, "WebAssign": threshold_table[14], "Exam marks": threshold_table[15]},  
           "F": {"Points": threshold_table[16], "Below P": 1000, "WebAssign": 0}
           # yes, I'm using 1000 as a threshold. I'm a teacher, not a programmer...
          }
    
    if individual.Marks.Score >= threshold['A']["Points"] and individual.Marks.NumberOfNs <= threshold["A"]["Below P"] and individual.WebAssign >= threshold["A"]["WebAssign"] and individual.Exam >= threshold["A"]["Exam marks"]:
        return "A"
    elif individual.Marks.Score >= threshold['B']["Points"] and individual.Marks.NumberOfNs <= threshold["B"]["Below P"] and individual.WebAssign >= threshold["B"]["WebAssign"] and individual.Exam >= threshold["B"]["Exam marks"]:
        return "B"
    elif individual.Marks.Score >= threshold['C']["Points"] and individual.Marks.NumberOfNs <= threshold["C"]["Below P"] and individual.WebAssign >= threshold["C"]["WebAssign"] and individual.Exam >= threshold["C"]["Exam marks"]:
        return "C"
    elif individual.Marks.Score >= threshold['D']["Points"] and individual.Marks.NumberOfNs <= threshold["D"]["Below P"] and individual.WebAssign >= threshold["D"]["WebAssign"] and individual.Exam >= threshold["D"]["Exam marks"]:
        return "D"
    elif individual.Marks.Score >= threshold['F']["Points"] and individual.Marks.NumberOfNs <= threshold["F"]["Below P"] and individual.WebAssign >= threshold["F"]["WebAssign"]:
        return "F"
    else:
        return "F"
        #return "None of the above?!?"


Here, we feed our data into an HTML template and use *weasyprint* to convert the HTML into a pretty PDF.

In [3]:
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML
from datetime import date

def print_the_report_for(individual):
    today = date.today().strftime("%B %d, %Y")
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template("template.html")
    template_vars = {
        "Name": individual.Name,
        "raw_data": individual.RawData.to_html(index=False, bold_rows=False, classes=["paleGrayRows"]),
        "score": individual.Marks.Score,
        "number_of_ns": individual.Marks.NumberOfNs,
        "webassign_grade": individual.WebAssign,
        "letter_grade": decide_letter_grade(individual),
        "date": today,
        "midterm_or_final": midtermOrFinal,
        "exam_marks": individual.Exam if individual.Exam < 1000 else "--",
        "grade_threshold": threshold_html_table,
        "extra_text": extra_text if extra_text else ""
    }

#    print(individual)
    output = template.render(template_vars)
    HTML(string=output).write_pdf("./reports/"+individual.Name+".pdf",stylesheets=["./report.css"])
   

Read in the data from the Blackboard exported CSV. 


**Future note** if blackboard columns change, this will be rather broken.

If you're just playing with this, you can read *test.csv* for test data.

In [4]:
import pandas as pd

filename_change_if_youre_testing = "242-final.csv" # if course=="math242" else "243FINALGRADES.csv"

df = pd.read_csv(filename_change_if_youre_testing)

math144_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 
               'F1', 'F2', 
               'L1', 'L2',
               'D1', 'DC1', 'D2', 'D3', 'DC2', 'DC3',
               'DA1', 'DA2', 'DA3', 'DA4', 'DA5', 'DA6', 'DA7',
               'EL1', 'EL2', 'EL3', 'EL4', 'EL5',
               'INT1', 'INT2', 'INT3', 'INT4', 'INT5', 'INT6',        
               'IT1', 'IT2', 'WebAssign']

math242_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 'Course ID',
               'F1', 'F2', 
               'L1', 'L2',
               'D1', 'D2', 'DC1', 'DC2', 'DC3', 'DC4', 'DC5',
               'DA1', 'DA2', 'DA3', 'DA4', 'DA5', 'DA6',
               'INT1', 'INT2', 'INT3', 'INT4', 'INT5',       
               'IA1', 'IA2', 'IA3', 'IA4',
               'IF1', 'IF2', 'WebAssign', 'Exam marks']

math243_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 
               'V1', 'V2', 'V3', 'V4', 'V5',
               'F1', 'F2', 'F3', 'F4', 
               'INT1', 'INT2', 'INT3', 'INT4', 'INT5', 
               'A1', 'A2', 
               'C1', 'C2', 'C3', 'C4', 'C5',
               'S1', 'S2', 'S3', 'S4', 'S5', 
               'P1', 'P2', 'P3', 'WebAssign']

math123_better_cols = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 'Course ID',
               'A1', 'A2',
               'T1', 'T2', 'T3', 'T4', 'T5',
               'G1', 'G2', 'G3',
               'I1', 'I2',
               'AT1', 'AT2', 'AT3', 'AT4', 'AT5',
               'O1', 'O2', 'O3',
               'V1', 'V2', 'V3',   
               'C2', 'C2', 'C3', 'C4', 'C5', 'WebAssign']

better_cols = math242_better_cols # if course=="math242" else math243_better_cols

df.columns = better_cols

## Make a better index column and dump everything not grade-related
df['Name'] = df['Last Name'] + ', ' + df['First Name']
df.drop(['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 'Course ID'], axis=1, inplace=True)
df.set_index("Name", inplace=True)




/var/folders/w_/wphcyvv15xv9nptrnpt5z0080000gn/T/ipykernel_89864/1865677300.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Define the tools for reading the P, PM, and empty strings into their corresponding numeric values

In [5]:
def sgn_to_number(sgn):    
    ## remove any emojis and space:
    try:
        sgn = sgn.replace("🎉","").strip()
        if (sgn == "PM"):
            return 2
        elif (sgn == "P"):
            return 1
        else:
            return 0
    except AttributeError:
        return 0;  ## it was 'nan' data type.
    
def marks_to_score(marks):
    
    # this needs to be a dictionary, not a Marks namedtuple
    return {'score' : sgn_to_number(marks), 
                'number of ns' : 1 if marks=="N" else 0}

def summative(marks):
    score_so_far = 0
    ns_so_far = 0
    for cell in marks:
        try: 
            score_so_far += cell['score']
            ns_so_far += cell['number of ns']
        except:
            pass
    return Marks(Score=score_so_far, NumberOfNs=ns_so_far)

Iterate over the dataset, sum marks, assign webassign grade. 

Note: Iterating over a *pandas* DataFrame is a bad idea for large datasets. If you're a student (not me), learn to do this differently. For this use case of no more that 40 records, I'm fine with performance loss.

In [6]:
class_scores = list()
for index, row in df.iterrows():
    raw = pd.DataFrame({'Standard':row.index, 'Marks':row.values})
    raw = raw.dropna()
    class_scores.append( Individual(Name=index, 
                                    Marks=summative(row.apply(marks_to_score)), 
                                    WebAssign=row.loc['WebAssign'], 
                                    Exam=int(float(row.loc['Exam marks'].replace(",",""))),
                                    RawData=raw) )

In [9]:

#for individual in class_scores:

# Convert the list of namedtuples to a DataFrame
outputDf = pd.DataFrame(class_scores, columns=Individual._fields)
outputDf['LetterGrade'] = outputDf.apply(decide_letter_grade, axis=1)

# Write the DataFrame to an Excel file
outputDf.to_excel('finalGrades.xlsx', index=False)

# for score in class_scores: 
#     print_the_report_for(score)